---
##**¡Los Juegos del Hambre!**
---
Autor: Juan Valverde Santiago

# Bienvenido a este puzzle.

En este juego, podrás programar el comportamiento de dos tipos de agente y verlos reflejados en una batalla entre los dos tipos. Al final, sólo quedará un agente ganador.

Para realizar esta tarea, sigue los pasos que se indican en cada sección.

Pero primero, deberás guardarte una copia de este cuaderno, pues no puedes guardar cambios en el original.

# Descarga e importación

El siguiente bloque obtiene los archivos necesarios para la correcta ejecución del cuaderno.

Simplemente, ejecuta el bloque sin modificar nada.

In [1]:
%%capture
!mkdir v002
!mkdir images
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Agent.py' -O v002/Agent.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment.py' -O v002/Enviroment.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/__init__.py' -O v002/__init__.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment_with_agents.py' -O v002/Enviroment_with_agents.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/InOut_Simple_Laberinth.py' -O v002/InOut_Simple_Laberinth.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelNoTomato.bmp' -O images/PixelNoTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelTomato.bmp' -O images/PixelTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/face1_borders.bmp' -O images/face1_borders.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival-CLIPS/raw/main/rules_to_clips_v3.py' -O rules_to_clips_v3.py

!pip install clipspy

from v002 import *
from rules_to_clips_v3 import *

# Escritura de las reglas

En este bloque debes trabajar. Aquí es donde debes escribir las reglas CLIPS y el nombre del agente.

Tu objetivo será crear un comportamiento para que los agentes peleen en un laberinto. No modifiques nombres de variables, céntrate en escribir las reglas que desees en la variable file.

Las percepciones del agente son los muros que le rodean y los objetos que encuentra, las opciones que puedes usar siguen la siguiente forma:
*   (walls \$? front \$?)
*   (not (walls \$? back \$?))
*   (available_object exit ?x)

Siendo los posibles muros front, back, left y right, que corresponden con los muros de delante, detrás, izquierda y derecha, respectivamente.

Y siendo los posibles objetos que encuentra el agente los siguientes:
*   exit
*   food type 1
*   agent

Por otro lado, las acciones que puede realizar el agente son las siguientes:
*   "self.move_forward()"
*   "self.turn_left()"
*   "self.turn_right()"
*   Funciones de objeto

Estas acciones deben añadirse en el hecho "pending_function_calls", como puedes ver en las reglas de ejemplo. En la causa de la regla se introduce el hecho en una variable; mientras que, en el consecuente se elimina el hecho y se vuelve a crear añadiendo las acciones que quieres que realice tu agente. En cuanto a las funciones de objeto, puedes observar la regla 4 de ejemplo llamada "salir" en la que se realiza la función de salida del laberinto.

Ten en cuenta que las acciones del agente consumen movimientos, y el número de movimientos está limitado.


Por último, también debes tener en cuenta el movimiento del agente modificando el hecho pos, que refiere a la posición del agente; y el hecho ori, que refiere hacia donde mira el agente. Lo que debes saber para resolver este problema es la estructura que usan estos datos:
*   (pos 0 0)
*   (ori 1 0)

Al comenzar el puzzle, la posición del agente(sea cuál sea) será la posición (0,0). En este juego la primera coordenada se refiere a la posición del eje Y y la segunda coordenada al eje X.

En cuanto a la orientación del agente pueden ser las siguientes:
*   Hacia arriba ---> (1, 0)
*   Hacia abajo ---> (-1, 0)
*   Hacia la izquierda ---> (0, -1)
*   Hacia la derecha ---> (0, 1)

Conociendo estos datos, debes introducir las operaciones para que el agente tenga sus hechos de posición y orientación correctos en todo momento. Esto es importante porque cuando el agente pasé por la misma posición 5 veces, se activará en el entorno el hecho (caution_loop), usado en la regla de ejemplo llamada "bucle", que también puedes usar para programar el comportamiento del agente.


¡Suerte!

In [2]:
# TODO: Aquí debes escribir las reglas CLIPS para los agentes Runners

file = """
    (defrule girarIzqda
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (+ ?p1 ?v2) (- ?p2 ?v1)))
    (assert (ori ?v2 (* -1 ?v1)))
    (assert (pending_function_calls $?calls "self.turn_left()"))
    )

    (defrule haciaDelante
    (not (walls $? front $?))
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (+ ?p1 ?v1) (+ ?p2 ?v2)))
    (assert (ori ?v1 ?v2))
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )

    (defrule girarDcha
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (- ?p1 ?v2) (+ ?p2 ?v1)))
    (assert (ori (* -1 ?v2) ?v1))
    (assert (pending_function_calls $?calls "self.turn_right()"))
    )

    (defrule comer
    (available_object food type 1 ?y)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls ?y))
    )
"""

with open("rules_run.CLP", "w") as txt:
    print(format(file), file=txt)

rules_run = "rules_run.CLP"

El siguiente bloque es un duplicado del anterior, en este puedes crear un diferente comportamiento para que haya dos tipos de agentes.
En el caso de que no quieras utilizar este segundo tipo de comportamiento, debes comentar aquellas líneas que tengan variables relativas a los agentes Punchers.

In [3]:
# TODO: Aquí debes escribir las reglas CLIPS para los agentes Punchers
file = """
    (defrule giraDcha
    ?h<-(pending_function_calls $?calls)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    =>
    (retract ?h ?h_ori ?h_pos)
    (assert (pos (- ?p1 ?v2) (+ ?p2 ?v1)))
    (assert (ori (* -1 ?v2) ?v1))
    (assert (pending_function_calls $?calls "self.turn_right()"))
    )

    (defrule haciaDelante
    (not (walls $? front $?))
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (+ ?p1 ?v1) (+ ?p2 ?v2)))
    (assert (ori ?v1 ?v2))
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )

    (defrule giraIzqda
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos ?p1 ?p2))
    (assert (ori ?v2 (* -1 ?v1)))
    (assert (pending_function_calls $?calls "self.turn_left()"))
    )

    (defrule golpear
    (available_object agent ?z)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls ?z))
    )"""

with open("rules_punch.CLP", "w") as txt:
    print(format(file), file=txt)

rules_punch = "rules_punch.CLP"

# Funciones move

Los siguientes bloques corresponden con las funciones move de los dos tipos de agente. Simplemente, ejecuta los bloques de código.

**¡NO DEBES MODIFICARLOS!**

En el siguiente bloque se inicializan los tipos de agentes que vayas a utilizar.

In [4]:
runner = Agent_CLIPS(rules_run)
puncher = Agent_CLIPS(rules_punch)

El siguiente bloque son las funciones move de los agentes, no modificar.

In [5]:
def move_run(self, available_objects=None):

    # Actualizar el entorno CLIPS y obtener las interacciones del agente
    walls, objects = runner.agent_context(self)

    # Obtener las funciones y los objetos gracias al algoritmo
    functions, available_objects = algorithm(objects, runner.env, available_objects)

    # Ejecutar las funciones en pending_function_calls
    for j in functions:
        j = j.replace('"','')
        eval(j)

def move_punch(self, available_objects=None):

    # Actualizar el entorno CLIPS y obtener las interacciones del agente
    walls, objetos = puncher.agent_context(self)

    # Obtener las funciones y los objetos gracias al algoritmo
    functions, available_objects = algorithm(objetos, puncher.env, available_objects)

    # Ejecutar las funciones en pending_function_calls
    for j in functions:
        j = j.replace('"','')
        eval(j)

# Ejecución del puzzle

Ha llegado el momento de ejecutar el puzzle para comprobar el comportamiento de los dos tipos de agente.

En principio, no debes modificar nada. Aunque puedes modificar algún parámetro de la función Enviroment_with_agents, por ejemplo el primer parámetro es el tamaño del escenario.

También puedes modificar las variables del principio del bloque, que corresponden al número de agentes y la vida que tienen al comienzo del juego.

In [ ]:
num_agents = 8
hp = 150

reset_position(runner.env, runner.memory)
reset_position(puncher.env, puncher.memory)

lb1 = Enviroment_with_agents(10, max_moves_per_turn=7,
                                 plot_run='every epoch',
                                 move_protection=False, remove_walls_prob=0.5)
for i in range(int(num_agents/2)):
    lb1.create_agent('Runner' + str(i+1), move_run, life=hp)
    lb1.create_agent('Puncher' + str(i+1), move_punch, life=hp)

lb1.run()